In [1]:
key_words = ["a","about","above","after","again","against","all","am","an","and","any","are","aren't","as","at","be","because","been","before","being","below","between","both","but","by","can't","cannot","could","couldn't","did","didn't","do","does","doesn't","doing","don't","down","during","each","few","for","from","further","had","hadn't","has","hasn't","have","haven't","having","he","he'd","he'll","he's","her","here","here's","hers","herself","him","himself","his","how","how's","i","i'd","i'll","i'm","i've","if","in","into","is","isn't","it","it's","its","itself","let's","me","more","most","mustn't","my","myself","no","nor","not","of","off","on","once","only","or","other","ought","our","ours","ourselves","out","over","own","same","shan't","she","she'd","she'll","she's","should","shouldn't","so","some","such","than","that","that's","the","their","theirs","them","themselves","then","there","there's","these","they","they'd","they'll","they're","they've","this","those","through","to","too","under","until","up","very","was","wasn't","we","we'd","we'll","we're","we've","were","weren't","what","what's","when","when's","where","where's","which","while","who","who's","whom","why","why's","with","won't","would","wouldn't","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves"]

punctuation_pattern = r'[!\"#$%&\'()*+,-./:;<=>?@\[\]\^_`{|}~—]'

In [2]:
import pandas as pd
import re
import json


def addRowsToDataframe(df, row):

    df.loc[len(df)] = row

def checkKeyWords(string):

    for keyword in key_words:
        match = re.search(rf'^{keyword}$', string=string)
        if(match):
            return False
    
    return True

def createHTSDict(path):
    df = pd.read_json(path)
    columns_df = df.columns.tolist()
    final_dict = {}
    main_numbers_pattern = re.compile(r'^[\d]{4}')
    start = True
    current_main_hts = ['']
    previous_main_hts = ['']

    for index,row in df.iterrows():

        current_main_hts = main_numbers_pattern.findall(row['htsno'])

        if start:
            previous_main_hts = current_main_hts
            final_dict[current_main_hts[0]] = pd.DataFrame(columns=columns_df)
            addRowsToDataframe(final_dict[current_main_hts[0]], row)

        if len(current_main_hts) == 0:
            current_main_hts = previous_main_hts

        if (previous_main_hts[0] != current_main_hts[0]) and (len(current_main_hts)):
            final_dict[current_main_hts[0]] = pd.DataFrame(columns=columns_df)
            addRowsToDataframe(final_dict[current_main_hts[0]], row)
            previous_main_hts = current_main_hts  
        elif start == False:
            addRowsToDataframe(final_dict[current_main_hts[0]], row)
        start = False

    return final_dict

def writeFiles(HTS_dict, path_hts, path_strings):
    string_dict = {}
    file_dict = {}

    for key,df in HTS_dict.items():
        
        file_dict[key] = f'{path_hts}/{key}.json'

        df.to_json(file_dict[key], orient='records')

        for row in df.iterrows():
            desc = re.sub(pattern=punctuation_pattern, repl='', string=row[1]['description']).lower()
            
            array_string = desc.split()
            if(len(array_string) > 0):
                for string in array_string:
                    if(string in string_dict): 
                        string_dict[string].append(key)
                        string_dict[string] = list(set(string_dict[string]))
                    else:
                        if(checkKeyWords(string)): 
                            string_dict[string] = []
                            string_dict[string].append(key)
                            string_dict[string] = list(set(string_dict[string]))
                            
        file_path = 'string_dict.json'

        with open(f'{path_strings}{file_path}', 'w') as json_file:
            json.dump(string_dict, json_file, indent=4)



In [3]:
writeFiles(createHTSDict('../db_hts/htsdata/htsdata.json'), 'test_files/', 'test_string_dict/')